In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df_rm = pd.read_csv('../data/03_GameSpy_RollingMeans.csv', index_col=0)
df = pd.read_csv('../data/02_GameSpy_Scrapping_MergeAllDataSet.csv', index_col=0)

# Ajout d'autres données prédictives

**Est-ce que le jeu fait partie d'une série ?**

In [3]:
#Fonction qui prend en entrée une url et qui retourne les éléments des noms présents dans les classes "colonnes"
def extract_series(url):
    
        names = url.find_all("div",class_="colonnes")
        
        return names

#Application de la fonction créée précédemment sur l'url suivante et récupération des éléments des noms présents sur la page
url_scrap = "https://fr.wikipedia.org/wiki/Liste_de_s%C3%A9ries_de_jeux_vid%C3%A9o"
url_page = requests.get(url_scrap)
parsed_url_page = BeautifulSoup(url_page.content, 'html.parser')
data_full=extract_series(parsed_url_page)

#Boucle qui pour chaque élément scrapé, extrait le texte c'est à dire les noms des jeux faisant partie d'une série
clean_names=[]
for i in data_full:
    clean_names.append(i.text)

#Boucle qui pour chaque nom de jeu va remplacer les caractères "\n"
clean_names2=[]
for name in clean_names:
    clean_names2.append(name.replace("\n",","))

#Boucle qui pour chaque nom de jeu va supprimer le caractère récupéré en trop avant et après le nom du jeu
clean_names3=[]
for element in clean_names2:
    clean_names3.append(element[1:-1])

#Boucle qui va récupérer une liste des noms de jeux entre chaque virgule
clean_names4=[]
for element in clean_names3:
    clean_names4.append(element.split(","))

#Boucle qui va reconstituer une liste propre des noms de jeux faisant partie d'une série, séparés par une virgule
fat_list=[]
for sublist in clean_names4:
    for element in sublist:
        fat_list.append(element)

#Transformation de la liste obtenue en série et retrait des noms avec accent
fat_list=pd.Series(fat_list)
fat_list=fat_list.replace(to_replace="Pokémon",value="Pokemon")

#Création d'une chaine de caractère avec les noms des jeux à la suite les uns des autres et séparés par des pipes
chaine = '|'.join(fat_list.values)
r=re.compile(chaine)

#Création d'une liste vide
masked=[]

#Boucle qui parcourt tous les noms des jeux présents dans le dataset et qui va enrichir la liste précédemment créée
#par 1 si le nom match avec un des jeux scrapés et 0 sinon
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)

#Création de la nouvelle colonne is_serie pour identifier les jeux faisant partie d'une série
df_rm["is_serie"]=masked

**Est-ce que le jeu fait partie d'une des séries les plus vendues ?**

In [4]:
#Fonction qui prend en entrée une url et qui retourne les noms des jeux présents dans les classes "mw-parser-output"
def extract_top_series(url):
    
    li_top_series=[]
    
    names = url.find_all("div",class_="mw-parser-output")
    
    for element in names:
        li=element.find_all("i")
            
    for i in li:
        li_top_series.append(i.text)
    
    return li_top_series

#Application de la fonction créée précédemment sur l'url suivante et récupération des noms de jeux présents sur la page
url_scrap = "https://fr.wikipedia.org/wiki/Liste_des_s%C3%A9ries_de_jeux_vid%C3%A9o_les_plus_vendues"
url_page = requests.get(url_scrap)
parsed_url_page = BeautifulSoup(url_page.content, 'html.parser')
data_full=extract_top_series(parsed_url_page)

#Transformation de la liste obtenue en série et retrait des noms avec accent
data_full[15]="Pokemon"

data_full=pd.Series(data_full)

#Création d'une chaine de caractère avec les noms des jeux à la suite les uns des autres et séparés par des pipes
chaine = '|'.join(data_full.values)
r=re.compile(chaine)

#Création d'une liste vide
masked=[]

#Boucle qui parcourt tous les noms des jeux présents dans le dataset et qui va enrichir la liste précédemment créée
#par 1 si le nom match avec un des jeux scrapés et 0 sinon
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)

#Création de la nouvelle colonne is_serie pour identifier les jeux faisant partie d'une série les plus vendues
df_rm["is_top_serie"]=masked

**Est-ce que le jeu a été lançé par un des studios du top 20 ?**

In [6]:
#Remplacement des valeurs contenant "Ubisoft" pour que les différents noms des filliles soient identifiés sous le même nom 
for studio in df_rm["Studio"]:
    if "Ubisoft" in str(studio):
        df_rm=df_rm.replace(to_replace=studio,value="Ubisoft")

#Uniformisation des valeurs abrégées et non abrégées de "Electronic Arts" sous l'abréviation "EA"
for studio in df_rm["Studio"]:
    if "EA" in str(studio):
        df_rm=df_rm.replace(to_replace=studio,value="EA")
    elif "Electronic Arts" in str(studio):
        df_rm=df_rm.replace(to_replace=studio,value="EA")
        
#Groupby sur les ventes globales par studio
top_studio=df_rm.groupby("Studio").agg({"Global_Sales":sum}).reset_index().sort_values(by="Global_Sales",ascending=False)

#Selection du top 20
top_studio=top_studio.head(20)

liste_top=top_studio["Studio"]

#Création d'une chaine de caratères avec tous les noms du top 20 studios séparés par des pipes
chaine = '|'.join(liste_top.values)
r=re.compile(chaine)

#Création d'une liste vide
masked=[]

#Boucle qui parcourt tous les noms des studios présents dans df et qui va enrichir la liste précédemment créée
#par 1 si le studio match avec un des studios top 20 et 0 sinon
for studio in df_rm["Studio"]:
    if r.match(str(studio)):
        masked.append(1)
    else:
        masked.append(0)

#Création de la nouvelle colonne is_top_studio pour indentifier les jeux rattachés au studio du top 20
df_rm["is_top_studio"]=masked

**Est-ce que le jeu a été cité au festival E3 ?**

In [7]:
#Regroupement des urls en fonction de leur format de page

link_e3_before2009=["https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2000",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2001",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2002",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2003",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2004",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2005",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2006",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2009"]

link_e3_after2009=["https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2010",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2011",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2012",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2013",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2014",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2015"]

link_e3_2016="https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2016"

In [8]:
li_final=[]

#Boucles qui vont parcourir chaque url des groupes d'url créés précédemment qui vont récupérer pour chaque url
#les noms des jeux cités sur la page et les ajouter les uns à la suite des autres dans la liste vide créée au dessus

#Urls sur le festival E3 avant 2009
for url in link_e3_before2009:
    url_scrap = str(url)
    r = requests.get(url_scrap)
    soup = BeautifulSoup(r.content, 'html.parser')

    div = soup.find_all("div",class_="mw-body-content mw-content-ltr")
    

    for element in div:
        li=element.find_all("i")
                
        for i in li:
            li_final.append(i.text)

#Urls sur le festival E3 après 2009
for url in link_e3_after2009:
    url_scrap = str(url)
    r = requests.get(url_scrap)
    soup = BeautifulSoup(r.content, 'html.parser')

    div = soup.find_all("table",class_="wikitable")
    

    for element in div:
        li=element.find_all("i")
                
        for i in li:
            li_final.append(i.text)
            
#Urls sur le festival E3 en 2016           
url_scrap = link_e3_2016
r = requests.get(url_scrap)
soup = BeautifulSoup(r.content, 'html.parser')

div = soup.find_all("table",{"role":True})
    
for element in div:
    li=element.find_all("i")
                
    for i in li:
        li_final.append(i.text)

#Transformation de la liste obtenue en série
li_final=pd.Series(li_final)

#Création d'une chaine de caratères avec tous les noms des jeux séparés par des pipes
chaine = '|'.join(li_final.values)
r=re.compile(chaine)

masked=[]

#Boucle qui parcourt tous les noms des jeux présents dans le datset et qui va enrichir la liste précédemment créée
#par 1 si le jeu match avec un des jeux cités au festival E3 et 0 sinon
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)

#Création de la nouvelle colonne is_e3 pour indentifier les jeux cités au festival E3
df_rm["is_e3"]=masked

**Est-ce que le jeu a été lançé en même temps qu'une console ?**

In [9]:
#Fonction qui prend en entrée une url et qui retourne les noms des jeux présents dans les classes "mw-body-content mw-content-ltr"
def extract_games_launch(url):
    
    li_games_launch=[]
    
    names = url.find_all("div",class_="mw-body-content mw-content-ltr")
    
    for element in names:
        li=element.find_all("i")
            
    for i in li:
        if i.text not in li_games_launch:
            li_games_launch.append(i.text)
    
    return li_games_launch

#Application de la fonction créée précédemment sur l'url suivante et récupération des noms de jeux présents sur la page
url_scrap = "https://fr.wikipedia.org/wiki/Liste_de_jeux_au_lancement_de_consoles_de_jeux_vid%C3%A9o"
url_page = requests.get(url_scrap)
parsed_url_page = BeautifulSoup(url_page.content, 'html.parser')
data_full=extract_games_launch(parsed_url_page)

#Transformation de la liste obtenue en série
data_full=pd.Series(data_full)

#Création d'une chaine de caratères avec tous les noms des jeux séparés par des pipes
chaine = '|'.join(data_full.values)
r=re.compile(chaine)

masked=[]

#Boucle qui parcourt tous les noms des jeux présents dans le datset et qui va enrichir la liste précédemment créée
#par 1 si le jeu match avec un des jeux lancés en simultané d'une console et 0 sinon
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)
        
#Création de la nouvelle colonne is_launch_plateform_associated pour indentifier les jeux lancés en même temps qu'une console        
df_rm["is_launch_plateform_associated"]=masked

In [10]:
#Afficage du dataset et des nouvelles colonnes créées
df_rm

,Name,Platform,Year,Genre,Publisher,Studio,NA_Sales,EU_Sales,JP_Sales,Other_Sales,...,RM_Studio_reviews,RM_Licence,RM_Licence_score,RM_Licence_rate,RM_Licence_reviews,is_serie,is_top_serie,is_top_studio,is_e3,is_launch_plateform_associated
0,Wii Sports,Wii,2006,Sports,Nintendo,Nintendo,41.49,29.02,3.77,8.46,...,229.0,30.01,8.45,8.10,229.0,1,1,1,1,1
1,Super Mario Bros.,NES,1985,Platform,Nintendo,Nintendo,29.08,3.58,6.81,0.77,...,12.0,1.59,7.00,7.10,172.0,1,1,1,0,1
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,Nintendo,15.85,12.88,3.79,3.31,...,39.5,9.02,7.10,7.35,28.0,1,1,1,0,0
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,Nintendo,15.75,11.01,3.28,2.96,...,136.0,82.74,7.60,7.05,210.0,1,1,1,1,1
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,Nintendo,11.27,8.89,10.22,1.00,...,197.0,0.06,9.00,8.90,197.0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16533,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,Woody Woodpecker,0.01,0.00,0.00,0.00,...,1.0,0.11,2.00,4.00,1.0,0,0,0,0,0
16534,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,Atari,0.01,0.00,0.00,0.00,...,1.0,0.06,5.90,6.20,1.0,0,0,0,0,0
16535,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,Left Field Productions,0.00,0.00,0.00,0.00,...,1.0,0.10,2.00,2.00,1.0,0,0,0,0,0
16536,Know How 2,DS,2010,Puzzle,7G//AMES,InXile Entertainment,0.00,0.01,0.00,0.00,...,0.0,0.08,5.90,5.60,0.0,0,0,0,0,0


In [ ]:
#Export du dataset
df_rm.to_csv('../data/04_GameSpy_FinalDataset.csv')